In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import time
import torch
import random
import scipy.io
import numpy as np
from pyDOE import lhs
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams.update(mpl.rcParamsDefault)
np.set_printoptions(threshold=sys.maxsize)
plt.rcParams["figure.max_open_warning"] = 4

In [3]:
### load local packages
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), "../.."))

if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

from src.data.helmholtz import generate_dataset
from src.nn.dnn import DNN
from src.nn.pinn import PINN

## Global functions

In [4]:
if torch.cuda.is_available():
    """Cuda support"""
    print("cuda available")
    DEVICE = torch.device("cuda")
else:
    print("cuda not avail")
    DEVICE = torch.device("cpu")


def seed_torch(seed):
    """Seed initialization"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)


seed_torch(2341)
torch.cuda.empty_cache()

cuda available


## Data generation

In [5]:
nx, nt, t0, x0, X_r_train, X_u_train, u_train, Exact0, lbc, ubc = generate_dataset(
    DEVICE
)

X_r shape: (25600, 2)
X_u shape: (200, 2)
u shape: (200, 1)


In [6]:
model = PINN(X_u_train, u_train, X_r_train, nx, nt, t0, x0, Exact0, device="cpu")

/home/ubuntu/miniconda3/envs/pytorch-2.3.1-corona-pinn/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


## Training

In [ ]:
%%time
model.train() # RBA

In [ ]:
%%time
model.train() # SA

In [ ]:
%%time
model.train() # Vanilla

In [10]:
# Prediction
Exact = Exact0
X, T = np.meshgrid(x0, t0)
X_star = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))
u_pred = model.predict(X_star)
U_pred = np.reshape(u_pred, (Exact.shape[1], Exact.shape[0])).T
l2_rel = np.linalg.norm(Exact.flatten() - U_pred.flatten(), 2) / np.linalg.norm(
    Exact.flatten(), 2
)
print("L2:", l2_rel)

plt.figure(1)
plt.title("Abs Error u")
plt.xlabel("y")
plt.ylabel("x")
plt.imshow(np.abs(Exact - U_pred), aspect="auto", cmap="rainbow")
plt.colorbar()
plt.show()

AttributeError: 'numpy.ndarray' object has no attribute 'device'

In [ ]:
def load_data(Fluid_data, key):
    return Fluid_data[key]


Fluid_data = scipy.io.loadmat(path)

# Load data
fluid = load_data(Fluid_data, "Fluid_training")
interface = load_data(Fluid_data, "Solid_interface")
solid = load_data(Fluid_data, "Solid_interface")

In [ ]:
INFO:src.utils.logger:MODEL_PATH /home/ubuntu/afrah/code/pinn_fsi_ibm/checkpoints/fsi/2024-10-15_17-36-49-818397/model.pth
INFO:src.utils.logger:model loaded from /home/ubuntu/afrah/code/pinn_fsi_ibm/checkpoints/fsi/2024-10-15_17-36-49-818397/model.pth
INFO:src.utils.logger:problem: fsi
INFO:src.utils.logger:dataset_path:  , None
INFO:src.utils.logger:batch size:  , 100
INFO:src.utils.logger:network:  , [3, 400, 400, 400, 5]
INFO:src.utils.logger:term loss weights :  , None
INFO:src.utils.logger:method:  , RBA
INFO:src.utils.logger:solver:  , xsig
INFO:src.utils.logger:number of iterations:  , 50001
INFO:src.utils.logger:RelL2_U%  : 3.294e+01 
INFO:src.utils.logger:RelL2_V%  : 4.238e+01 
INFO:src.utils.logger:RelL2_P%  : 7.043e+01 
INFO:src.utils.logger:RelL2_Fx%  : 1.000e+02 
INFO:src.utils.logger:RelL2_Fx%  : 1.000e+02 
INFO:src.utils.logger:Final loss left: 1.683178e-03
INFO:src.utils.logger:Final loss right: 4.221339e-03
INFO:src.utils.logger:Final loss bottom: 1.297961e-05
INFO:src.utils.logger:Final loss up: 9.127887e-03
INFO:src.utils.logger:Final loss fluid_points: 7.133770e-03
INFO:src.utils.logger:Final loss initial: 2.833534e-02
INFO:src.utils.logger:Final loss fluid: 2.689436e-03
INFO:src.utils.logger:final loss sum: 5.32e-02
INFO:src.utils.logger:******************************

INFO:src.utils.logger:MODEL_PATH /home/ubuntu/afrah/code/pinn_fsi_ibm/checkpoints/fsi/2024-10-15_18-20-15-632977/model.pth
INFO:src.utils.logger:model loaded from /home/ubuntu/afrah/code/pinn_fsi_ibm/checkpoints/fsi/2024-10-15_18-20-15-632977/model.pth
INFO:src.utils.logger:problem: fsi
INFO:src.utils.logger:dataset_path:  , None
INFO:src.utils.logger:batch size:  , 100
INFO:src.utils.logger:network:  , [3, 200, 200, 200, 200, 5]
INFO:src.utils.logger:term loss weights :  , None
INFO:src.utils.logger:method:  , Fixed
INFO:src.utils.logger:solver:  , xsig
INFO:src.utils.logger:number of iterations:  , 10001
INFO:src.utils.logger:RelL2_U%  : 1.409e+01 
INFO:src.utils.logger:RelL2_V%  : 1.940e+01 
INFO:src.utils.logger:RelL2_P%  : 4.182e+01 
INFO:src.utils.logger:RelL2_Fx%  : 1.000e+02 
INFO:src.utils.logger:RelL2_Fx%  : 1.000e+02 
INFO:src.utils.logger:Final loss left: 2.806298e-03
INFO:src.utils.logger:Final loss right: 3.064817e-04
INFO:src.utils.logger:Final loss bottom: 1.725764e-05
INFO:src.utils.logger:Final loss up: 8.084860e-04
INFO:src.utils.logger:Final loss fluid_points: 3.315340e-03
INFO:src.utils.logger:Final loss initial: 3.685336e-03
INFO:src.utils.logger:Final loss fluid: 9.500139e-01
INFO:src.utils.logger:final loss sum: 9.61e-01
INFO:src.utils.logger:******************************

INFO:src.utils.logger:MODEL_PATH /home/ubuntu/afrah/code/pinn_fsi_ibm/checkpoints/fsi/2024-10-15_17-37-33-081291/model.pth
INFO:src.utils.logger:model loaded from /home/ubuntu/afrah/code/pinn_fsi_ibm/checkpoints/fsi/2024-10-15_17-37-33-081291/model.pth
INFO:src.utils.logger:problem: fsi
INFO:src.utils.logger:dataset_path:  , None
INFO:src.utils.logger:batch size:  , 100
INFO:src.utils.logger:network:  , [3, 400, 400, 400, 5]
INFO:src.utils.logger:term loss weights :  , None
INFO:src.utils.logger:method:  , SA
INFO:src.utils.logger:solver:  , xsig
INFO:src.utils.logger:number of iterations:  , 50001
INFO:src.utils.logger:RelL2_U%  : 3.118e+01 
INFO:src.utils.logger:RelL2_V%  : 3.967e+01 
INFO:src.utils.logger:RelL2_P%  : 6.425e+01 
INFO:src.utils.logger:RelL2_Fx%  : 1.000e+02 
INFO:src.utils.logger:RelL2_Fx%  : 1.000e+02 
INFO:src.utils.logger:Final loss left: 2.478776e-03
INFO:src.utils.logger:Final loss right: 2.160316e-03
INFO:src.utils.logger:Final loss bottom: 4.728580e-05
INFO:src.utils.logger:Final loss up: 3.277555e-03
INFO:src.utils.logger:Final loss fluid_points: 1.019266e-02
INFO:src.utils.logger:Final loss initial: 4.375745e-02
INFO:src.utils.logger:Final loss fluid: 4.457249e-03
INFO:src.utils.logger:final loss sum: 6.64e-02
INFO:src.utils.logger:******************************

INFO:src.utils.logger:MODEL_PATH /home/ubuntu/afrah/code/pinn_fsi_ibm/checkpoints/fsi/2024-10-15_17-37-23-782676/model.pth
INFO:src.utils.logger:model loaded from /home/ubuntu/afrah/code/pinn_fsi_ibm/checkpoints/fsi/2024-10-15_17-37-23-782676/model.pth
INFO:src.utils.logger:problem: fsi
INFO:src.utils.logger:dataset_path:  , None
INFO:src.utils.logger:batch size:  , 100
INFO:src.utils.logger:network:  , [3, 400, 400, 400, 5]
INFO:src.utils.logger:term loss weights :  , None
INFO:src.utils.logger:method:  , grad_stat
INFO:src.utils.logger:solver:  , xsig
INFO:src.utils.logger:number of iterations:  , 50001
INFO:src.utils.logger:RelL2_U%  : 1.854e+01 
INFO:src.utils.logger:RelL2_V%  : 2.666e+01 
INFO:src.utils.logger:RelL2_P%  : 4.321e+01 
INFO:src.utils.logger:RelL2_Fx%  : 1.000e+02 
INFO:src.utils.logger:RelL2_Fx%  : 1.000e+02 
INFO:src.utils.logger:Final loss left: 1.840389e-04
INFO:src.utils.logger:Final loss right: 9.571897e-05
INFO:src.utils.logger:Final loss bottom: 1.398892e-07
INFO:src.utils.logger:Final loss up: 5.822193e-03
INFO:src.utils.logger:Final loss fluid_points: 2.536351e-03
INFO:src.utils.logger:Final loss initial: 7.254026e-03
INFO:src.utils.logger:Final loss fluid: 8.639111e-01
INFO:src.utils.logger:final loss sum: 8.80e-01
INFO:src.utils.logger:******************************

INFO:src.utils.logger:file directory:
INFO:src.utils.logger:/home/ubuntu/afrah/code/pinn_fsi_ibm/result/fsi/2024-10-15_16-23-57-095513
